In [1]:
import numpy as np

In [2]:
params = np.random.uniform(low =-50, high=150, size=20)
params

array([  2.10765569, 110.44483913, -13.66585072,  88.15961983,
         0.51923735,  93.90104613, -27.83028434,  27.15113248,
       128.6031137 ,   2.98600289, 148.99545196,  29.76906547,
       140.6226831 ,  24.92445399,  28.70714817, 115.19625712,
        72.64920264,  42.65152772, 135.04727475,  83.32600314])

In [10]:
params_2 = np.random.uniform(low =-50, high=150, size=20)
params_2

array([ 97.90587899, -30.54347373, 107.06069995, -24.87767714,
       -13.12574342, -40.45456006,  13.59020207, 144.38601142,
       -48.88340416, -46.21046313,  -2.44545951,  20.7496684 ,
        52.14131184,  55.01623051, -21.94195487, 115.82113461,
        64.76028566,  57.74994078,  54.26420125,  68.1472703 ])

In [3]:
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

params = np.round(params, 2)
params

array([150.  , -28.83,   0.  ,  88.16,   0.52,  93.9 , -27.83,  27.15,
       128.6 ,   2.99, 149.  ,  29.77, 140.62,  24.92,  28.71, 115.2 ,
        72.65,  42.65, 135.05,  83.33])

In [11]:
params_2[0] = params_2.max() + 1
params_2[1] = params_2.min() - 1
params_2[2] = 0

params_2 = np.round(params_2, 2)
params_2

array([145.39, -49.88,   0.  , -24.88, -13.13, -40.45,  13.59, 144.39,
       -48.88, -46.21,  -2.45,  20.75,  52.14,  55.02, -21.94, 115.82,
        64.76,  57.75,  54.26,  68.15])

In [12]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

In [13]:
def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1

    # quantize the parameters
    quantized = clamp(np.array(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale



def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits - 1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale
    
def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q* scale

In [14]:
def quantization_error(params: np.array, params_q: np.array):
    # Calculate the MSE
    return np.mean((params - params_q)**2)

In [7]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original: \n{np.round(params, 2)}')

print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)

print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original: 
[150.   -28.83   0.    88.16   0.52  93.9  -27.83  27.15 128.6    2.99
 149.    29.77 140.62  24.92  28.71 115.2   72.65  42.65 135.05  83.33]
Asymmetric scale: 0.7012941176470587, zero: 41.0
[254   0  41 166  41 174   1  79 224  45 253  83 241  76  81 205 144 101
 233 159]
Symmetric scale: 1.1811023622047243
[127 -24   0  75   0  80 -24  23 109   3 126  25 119  21  24  98  62  36
 114  71]


In [15]:
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)
(symmetric_q2, symmetric_scale2) = symmetric_quantization(params_2, 8)

print(f'Original: \n{np.round(params, 2)}')

print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

print(f'Original: \n{np.round(params_2, 2)}')
print(f'Symmetric scale: {symmetric_scale2}')
print(symmetric_q2)

#print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
#print(asymmetric_q)

#print(f'Symmetric scale: {symmetric_scale}')
#print(symmetric_q)

Original: 
[150.   -28.83   0.    88.16   0.52  93.9  -27.83  27.15 128.6    2.99
 149.    29.77 140.62  24.92  28.71 115.2   72.65  42.65 135.05  83.33]
Symmetric scale: 1.1811023622047243
[127 -24   0  75   0  80 -24  23 109   3 126  25 119  21  24  98  62  36
 114  71]
Original: 
[145.39 -49.88   0.   -24.88 -13.13 -40.45  13.59 144.39 -48.88 -46.21
  -2.45  20.75  52.14  55.02 -21.94 115.82  64.76  57.75  54.26  68.15]
Symmetric scale: 1.1448031496062991
[127 -44   0 -22 -11 -35  12 126 -43 -40  -2  18  46  48 -19 101  57  50
  47  60]


In [16]:
sum = params + params_2
print(f'Original Sum: \n{np.round(sum, 2)}')

sumq = symmetric_q + symmetric_q2
print(f'Quantised Sum: \n{np.round(sumq, 2)}')

Original Sum: 
[295.39 -78.71   0.    63.28 -12.61  53.45 -14.24 171.54  79.72 -43.22
 146.55  50.52 192.76  79.94   6.77 231.02 137.41 100.4  189.31 151.48]
Quantised Sum: 
[254 -68   0  53 -11  45 -12 149  66 -37 124  43 165  69   5 199 119  86
 161 131]


In [17]:
sum_dq = symmetric_dequantize(sumq, symmetric_scale)
sum_dq

array([300.        , -80.31496063,   0.        ,  62.5984252 ,
       -12.99212598,  53.1496063 , -14.17322835, 175.98425197,
        77.95275591, -43.7007874 , 146.45669291,  50.78740157,
       194.88188976,  81.49606299,   5.90551181, 235.03937008,
       140.5511811 , 101.57480315, 190.15748031, 154.72440945])

In [18]:
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(sum, sum_dq), 2)}')


   Symmetric error: 4.7


In [8]:
# Dequantize the parameters back to float 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original: \n{np.round(params, 2)}')

print(f'Dequantized asymmetric:')
print(params_deq_asymmetric)

print(f'Dequantized symmetric:')
print(params_deq_symmetric)

Original: 
[150.   -28.83   0.    88.16   0.52  93.9  -27.83  27.15 128.6    2.99
 149.    29.77 140.62  24.92  28.71 115.2   72.65  42.65 135.05  83.33]
Dequantized asymmetric:
[149.37564706 -28.75305882   0.          87.66176471   0.
  93.27211765 -28.05176471  26.64917647 128.33682353   2.80517647
 148.67435294  29.45435294 140.25882353  24.54529412  28.05176471
 115.01223529  72.23329412  42.07764706 134.64847059  82.75270588]
Dequantized symmetric:
[150.         -28.34645669   0.          88.58267717   0.
  94.48818898 -28.34645669  27.16535433 128.74015748   3.54330709
 148.81889764  29.52755906 140.5511811   24.80314961  28.34645669
 115.7480315   73.22834646  42.51968504 134.64566929  83.85826772]


In [9]:
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.18
   Symmetric error: 0.15
